In [1]:
from __future__ import print_function
from vowpalwabbit import pyvw

In [2]:
tag = {}
tag['CC'] = 1
tag['CD'] = 6
tag['DT'] = 7
tag['EX'] = 8
tag['FW'] = 9
tag['IN'] = 10
tag['JJ'] = 2
tag['JJR'] = 2
tag['JJS'] = 2
tag['MD'] = 11
tag['NN'] = 3
tag['NNP'] = 3
tag['NNPS'] = 3
tag['NNS'] = 3
tag['PDT'] = 12
tag['POS'] = 13
tag['PRP'] = 3
tag['PRP$'] = 3
tag['RB'] = 4
tag['RBR'] = 4
tag['RBS'] = 4
tag['RP'] = 14
tag['SYM'] = 15
tag['TO'] = 16
tag['UH'] = 17
tag['VB'] = 5
tag['VBD'] = 5
tag['VBG'] = 5
tag['VBN'] = 5
tag['VBP'] = 5
tag['VBZ'] = 5
tag['WDT'] = 5
tag['WP'] = 18
tag['WP$'] = 19
tag['WRB'] = 20

In [3]:
s = ''
punc = ['#', '$',"''",'(',')',',','.',':',"``"]
sentence = []
sentences = []
pos_tags = set([])
file = open('train.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences.append(sentence)
        sentence = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence.append(tup)

In [4]:
sentences[0][1][0]

10

In [4]:
class SequenceLabeler(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        # you must must must initialize the parent class
        # this will automatically store self.sch <- sch, self.vw <- vw
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        
        # set whatever options you want
        sch.set_options( sch.AUTO_HAMMING_LOSS | sch.AUTO_CONDITION_FEATURES )

    def _run(self, sentence):   # it's called _run to remind you that you shouldn't call it directly!
        output = []
        for n in range(len(sentence)):
            pos,word = sentence[n]
            # use "with...as..." to guarantee that the example is finished properly
            with self.vw.example({'w': [word]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'), (n-1, 'q')])
                output.append(pred)
        return output

## Customizing features


In [66]:
class SequenceLabeler2(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        #loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            prev_n_2 = output[n-2] if n > 1 else '<s>'
            posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            with self.vw.example({'w': [word],'p': [prevPred], 'r': [prev_n_2], 's':[prevWord]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'), (n-1, 'r'), (n,'s')])
                output.append(pred)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

In [67]:
vw = pyvw.vw("--search 20 --audit --quiet --search_task hook --ring_size 1024 -f pos_tagger.model")

In [68]:
sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)

In [ ]:
for i in range(1):
    print (i)
    sequenceLabeler2.learn(sentences[:1000])

0


In [9]:
temp=[]
for w in sentences[0]:
    temp.append((0,w[1]))
print (temp)

[(0, 'Confidence'), (0, 'in'), (0, 'the'), (0, 'pound'), (0, 'is'), (0, 'widely'), (0, 'expected'), (0, 'to'), (0, 'take'), (0, 'another'), (0, 'sharp'), (0, 'dive'), (0, 'if'), (0, 'trade'), (0, 'figures'), (0, 'for'), (0, 'September'), (0, 'due'), (0, 'for'), (0, 'release'), (0, 'tomorrow'), (0, 'fail'), (0, 'to'), (0, 'show'), (0, 'a'), (0, 'substantial'), (0, 'improvement'), (0, 'from'), (0, 'July'), (0, 'and'), (0, 'August'), (0, "'s"), (0, 'near-record'), (0, 'deficits')]


In [29]:
out = sequenceLabeler2.predict(temp)
print(out)
loss=0
c=0
for i in out:
    if i != sentences[0][c][0]:
        loss+=1
        print ("Predicted value at ",c," is ",i)
        print (sentences[0][c])
    c+=1



[10, 3, 10, 10, 3, 3, 5, 5, 7, 2, 3, 7, 10, 3, 13, 2, 5, 5, 2, 2, 3, 16, 3, 3]
Predicted value at  0  is  10
(3, 'Confidence')
Predicted value at  1  is  3
(10, 'in')
Predicted value at  2  is  10
(7, 'the')
Predicted value at  3  is  10
(3, 'pound')
Predicted value at  4  is  3
(5, 'is')
Predicted value at  5  is  3
(4, 'widely')
Predicted value at  7  is  5
(16, 'to')
Predicted value at  8  is  7
(5, 'take')
Predicted value at  9  is  2
(7, 'another')
Predicted value at  10  is  3
(2, 'sharp')
Predicted value at  11  is  7
(3, 'dive')
Predicted value at  14  is  13
(3, 'figures')
Predicted value at  15  is  2
(10, 'for')
Predicted value at  16  is  5
(3, 'September')
Predicted value at  17  is  5
(2, 'due')
Predicted value at  18  is  2
(10, 'for')
Predicted value at  19  is  2
(3, 'release')
Predicted value at  21  is  16
(5, 'fail')
Predicted value at  22  is  3
(16, 'to')
Predicted value at  23  is  3
(5, 'show')


In [30]:
sentence_test = []
sentences_test = []
pos_tags_test = set([])
file = open('test.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences_test.append(sentence_test)
        sentence_test = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags_test.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence_test.append(tup)

In [31]:
sentences_test[0]

[(3, 'Rockwell'),
 (3, 'International'),
 (3, 'Corp.'),
 (13, "'s"),
 (3, 'Tulsa'),
 (3, 'unit'),
 (5, 'said'),
 (3, 'it'),
 (5, 'signed'),
 (7, 'a'),
 (2, 'tentative'),
 (3, 'agreement'),
 (5, 'extending'),
 (3, 'its'),
 (3, 'contract'),
 (10, 'with'),
 (3, 'Boeing'),
 (3, 'Co.'),
 (16, 'to'),
 (5, 'provide'),
 (2, 'structural'),
 (3, 'parts'),
 (10, 'for'),
 (3, 'Boeing'),
 (13, "'s"),
 (6, '747'),
 (3, 'jetliners')]

In [ ]:

loss_t=[]
s=0
for sen in sentences_test:
    s+=1
    if(s%100==0):
        print(s)
    temp=[]
    for w in sen:
        temp.append((0,w[1]))
    out = sequenceLabeler2.predict(temp)
    #print(out)
    loss=0
    c=0
    for i in out:
        if i != sen[c][0]:
            loss+=1
            #print ("Predicted value at ",c," is ",i)
            #print (sentences[0][c])
        c+=1
    loss_t.append(loss/float(c))
    

print (sum(loss_t) / float(len(loss_t)))

100
200
300
